In [1]:
###el bueno
import scipy.io
import numpy
import torchtext
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchtext.vocab import Vectors, GloVe
import torch.nn.functional as F
import pdb
from copy import deepcopy
import pdb

In [2]:
##GRAND_FATHER
num_constants = 9
#Background
father_extension = torch.zeros(num_constants, num_constants)
father_extension[0,1] = 1
father_extension[0,2] = 1
father_extension[1,3] = 1
father_extension[1,4] = 1

mother_extension = torch.zeros(num_constants, num_constants)
mother_extension[8,0] = 1
mother_extension[2,5] = 1
mother_extension[2,6] = 1
mother_extension[5,7] = 1


#Intensional Predicates
aux_extension = torch.zeros(num_constants,num_constants)
target_extension = torch.zeros(num_constants,num_constants)

valuation_init = [Variable(father_extension), Variable(mother_extension), Variable(aux_extension), Variable(target_extension)]

num_predicates= len(valuation_init)
intensional_predicates=[2,3]
num_intensional_predicates = len(intensional_predicates)

#Target
target = torch.zeros(num_constants,num_constants)
target[8,1] =1 
target[8,2] =1 
target[0,3] =1 
target[0,4] =1 
target[0,5] =1 
target[0,6] =1 
target[2,7] =1 


In [3]:
print(valuation_init[0], target, valuation_init[3])

(Variable containing:
    0     1     1     0     0     0     0     0     0
    0     0     0     1     1     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
[torch.FloatTensor of size 9x9]
, 
    0     0     0     1     1     1     1     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     1     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     1     1     0  

In [12]:
def decoder_efficient(valuation, step, num_constants):
    
    ## Create valuation_new
    valuation_new = [deepcopy(valuation[0]),deepcopy(valuation[1]),Variable(torch.zeros(valuation[1].size())),Variable(torch.zeros(valuation[2].size()))]
                    
    ##Unifications
    rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
    rules_aux = rules_aux.repeat(num_predicates,1)
    embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)
    #embeddings_aux = F.dropout(embeddings_aux, p=.1,training=True)
    #rules_aux = F.dropout(rules_aux, p=.1, training=True)
    
    #unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
    
    unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
    unifs = torch.exp(-unifs)
    unifs_sum = torch.sum(unifs, 0)
    unifs= unifs/unifs_sum
    
    
    ###TODO I want to normalize across the rules!!
    #unifs = rbf(embeddings_aux, rules_aux).view(num_predicates,-1)
    #unifs = F.relu(unifs)
    #unifs_sum = torch.sum(unifs, 0)
    #unifs= unifs/unifs_sum
    #unifs = torch.min(unifs,Variable(torch.ones(unifs.size())))

    ##Get_Valuations
    for predicate in intensional_predicates:
        if valuation[predicate].size()[0] == 1:
            #pdb.set_trace()
            for s in range(num_constants):
                valuation_aux = Variable(torch.Tensor([0]))
                for body1 in range(num_predicates):
                    for body2 in range(num_predicates):
                        ## Get nums
                        if valuation[body1].size()[0] == 1:
                            if valuation[body2].size()[0] == 1:
                                num = torch.min(valuation[body1][0,s],valuation[body2][0,s])
                            else: 
                                num = torch.min(valuation[body1][0,:],valuation[body2][:,s])
                                num = torch.max(num)
                        else:
                            if valuation[body2].size()[0] == 1:
                                num = torch.min(valuation[body1][:,s],valuation[body2][0,s])
                                num = torch.max(num)
                            else:
                                num = torch.min(valuation[body1][s,:],valuation[body2][:,s])
                                #num = torch.min(valuation[body1][s,:],valuation[body2][:,o])
                                num = torch.max(num)

                        ## max across three rules
                        new = Variable(torch.Tensor([0]))
                        for rule in range(num_rules): 
                            unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                            new = torch.max(new,unif) 

                        num = num*new 
                        valuation_aux = torch.max(valuation_aux, num)
                        if valuation_aux.data[0] > 1:
                                pdb.set_trace()
                        if predicate == 3 and s==0 and num.data[0]>.3:
                            print('bodyyyyy0',body1, body2)
                        if predicate == 3 and s==2 and num.data[0]>.3:
                            print('bodyyyyy2',body1, body2)
                        if predicate == 3 and s==4 and num.data[0]>.3:
                            print('bodyyyyy4',body1, body2) 
                        #if predicate == 3 and s==0 and o==6 and num.data[0]>.3:
                            #print('bodyyyyy6',body1, body2)
                        #if predicate == 3 and s==0 and o==8 and num.data[0]>.3:
                            #print('bodyyyyy8',body1, body2)   
                valuation_new[predicate][0,s] = torch.max(valuation[predicate][0,s], valuation_aux) 
                
        else:
            for s in range(num_constants):
                for o in range(num_constants):
                    valuation_aux = Variable(torch.Tensor([0]))
                    for body1 in range(num_predicates):
                        for body2 in range(num_predicates):
                            ## Get nums
                            if valuation[body1].size()[0] == 1:
                                if valuation[body2].size()[0] == 1:
                                    num = torch.min(valuation[body1][0,s],valuation[body2][0,o])
                                else: 
                                    num = torch.min(valuation[body1][0,s],valuation[body2][s,o])
                                    #num = torch.max(num)
                            else:
                                if valuation[body2].size()[0] == 1:
                                    num = torch.min(valuation[body1][s,o],valuation[body2][0,o])
                                    #num = torch.max(num)
                                else: 
                                    num = torch.min(valuation[body1][s,:],valuation[body2][:,o])
                                    num = torch.max(num)
                                    num_aux = torch.min(valuation[body1][s,o],valuation[body2][s,o])
                                    num = torch.max(num, num_aux)

                            ## max across three rules
                            new = Variable(torch.Tensor([0]))
                            for rule in range(num_rules): 
                                unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                                new = torch.max(new,unif)
                                #could be amalgamate

                            num = num*new 
                            valuation_aux = torch.max(valuation_aux, num)

                            if predicate == 3 and s==0 and o==1 and num.data[0]>.001:
                                print('bodyyyyy1',body1, body2)
    
                    valuation_new[predicate][s,o] = torch.max(valuation[predicate][s,o], valuation_aux) 
                
    return valuation_new
           
def amalgamate(x,y):
    return x + y - x*y

def rbf(x,y):
    ans = (x-y)**2
    ans = torch.mean(ans,1)
    ans = torch.exp(-ans)
    return ans    
            
                

In [13]:
num_iters = 1
learning_rate = .1
steps = 4

num_feat=4
num_rules = 3
num_predicates= 4
intensional_predicates=[2,3]
num_intensional_predicates = len(intensional_predicates)

#embeddings = Variable(torch.rand(num_predicates, num_feat), requires_grad=True)
#rules = Variable(torch.rand(num_rules, num_feat*3), requires_grad=True)
#optimizer = torch.optim.Adam([embeddings,rules],lr=learning_rate)

embeddings = Variable(torch.eye(num_predicates), requires_grad=True)
#rules = Variable(torch.rand(num_rules, num_feat*3), requires_grad=True)
rule1 = torch.Tensor([0,0,0,1,0,0,1,0,0,0,1,0]).view(1,-1)
rule2 = torch.Tensor([0,0,1,0,1,0,0,0,1,0,0,0]).view(1,-1)
rule3 = torch.Tensor([0,0,1,0,0,1,0,0,0,1,0,0]).view(1,-1)
rules = Variable(torch.cat((rule1,rule2,rule3),0), requires_grad=True)

optimizer = torch.optim.Adam([rules, embeddings],lr=learning_rate)

criterion = torch.nn.BCELoss(size_average=False)


for epoch in range(num_iters):
    optimizer.zero_grad()
    
    valuation = valuation_init
    for step in range(steps):
        valuation = decoder_efficient(valuation, step, valuation[0].size()[1])
        print('step',step,'valuation3', valuation[3]) 
    loss = criterion(valuation[3],Variable(torch.Tensor(target)))
    
    print(epoch,'lossssssssssssssssssssssssssss',loss.data[0])
    loss.backward()
    optimizer.step()
    
    

Variable containing:
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
[torch.FloatTensor of size 9x9]

('bodyyyyy1', 0, 0)
('step', 0, 'valuation3', Variable containing:
1.00000e-02 *
  0.0000  4.7007  4.7007  4.7007  4.7007  1.1428  1.1428  0.0000  0.0000
  0.0000  0.0000  0.0000  4.7007  4.7007  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  4.7007  4.7007  4.7007  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.00

In [28]:
print(target)
print(embeddings)
print(rules)


    0     0     0     1     1     1     1     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     1     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0
    0     1     1     0     0     0     0     0     0
[torch.FloatTensor of size 9x9]

Variable containing:
 1  0  0  0
 0  1  0  0
 0  0  1  0
 0  0  0  1
[torch.FloatTensor of size 4x4]

Variable containing:

Columns 0 to 9 
-0.0862 -0.0862 -0.0862  1.2880 -0.0862 -0.0862  1.2880 -0.0862 -0.0862 -0.0862
-0.5699 -0.5699  1.2397  0.5087  1.2515 -0.1120 -0.1112 -0.1112  1.2513 -0.1115
-0.5337 -0.5337  1.1862  0.4706 -0.0603  1.4260 -0.0565 -0.0565 -0.0600  1.4258

Columns 10 to 11 
 1.2880 -0.0862
-0.1115 -0.1115
-0.0553 -0.0553
[torch.FloatTensor of size 3x12]


In [31]:

rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
rules_aux = rules_aux.repeat(num_predicates,1)
embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)


#unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
#print('aaaaa',unifs)
#print('oooo',F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1))

unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
unifs = torch.exp(-unifs)
unifs_sum = torch.sum(unifs, 0)
unifs= unifs/unifs_sum
print('finaaaal',unifs)


('finaaaal', Variable containing:
 0.0463  0.0309  0.0288  0.0495  0.9321  0.0232  0.0496  0.9318  0.0232
 0.0315  0.0330  0.0349  0.0559  0.0353  0.9293  0.0560  0.0355  0.9305
 0.0611  0.7939  0.8137  0.8152  0.0220  0.0197  0.8149  0.0221  0.0195
 0.8611  0.1422  0.1226  0.0794  0.0105  0.0277  0.0795  0.0105  0.0267
[torch.FloatTensor of size 4x9]
)


In [ ]:

input =torch.randn(5,7)
print(input)
print(F.dropout(input, p=.2,training=True))